In [1]:
import math
import os
import random
import re
import time
from collections import Counter
from contextlib import closing
from string import punctuation as enPunc

import gensim
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from gensim.models.word2vec import Word2Vec
from LAC import LAC
from opencc import OpenCC
from zhon.hanzi import punctuation as zhPunc

## Target Page : V 1.2 (Api version)

- input: 某taskId包含之網址
- output: 網頁重新排序後之列表
---------------------
0. (載入)預訓練詞向量
1. 爬蟲&資料預處理：爬網頁文章內容、分詞並計算tfidf
2. 排序模組：匯入詞向量並進行增量訓練，取得搜尋詞詞向量，取得文件向量並與搜尋詞比較相似性

In [22]:
user_name = 'csti-user'
file_dir = 'OSINT'

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 爬蟲&資料預處理

#### 爬蟲取得網頁內容文章

In [23]:
def __get_content_texts(google_search_rs):
    """爬蟲取得網頁內容文章
    
    :param google_search_rs: 特定taskID中所有url
    :return context: 網頁文章，以句子分開
    """
    articleLength, context = [], []
    for idx, url in enumerate(google_search_rs):
#         print(idx +1, url)
        try:
            with closing(requests.get(url, headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
            }, timeout=30, verify=False)) as resp:
                if resp.ok and resp.headers.get("Content-Type", "").startswith("text/html"):
                    soup = BeautifulSoup(resp.content.decode("utf-8", "ignore"), "lxml")
                    if soup.body:
                        for tree in soup.find_all([
                            "a", "audio", "button", "head", "iframe", "img",
                            "input", "link", "meta", "noscript", "object", "script",
                            "select", "source", "style", "textarea", "title", "video"
                        ]):
                            tree.extract()
                        if soup.body.find("main"):
                            texts = [
                                re.sub(r"[\xa0\r\n\t\f\v ]+", " ", text.strip())
                                for text in soup.body.find("main").get_text().splitlines()
                                if re.search(r"\w", text)
                            ]
                        else:
                            texts = [
                                re.sub(r"[\xa0\r\n\t\f\v ]+", " ", text.strip())
                                for text in soup.body.get_text().splitlines()
                                if re.search(r"\w", text)
                            ]
            context.append(texts)
        except Exception as e:
            context.append([""])
            print(e)
        
        length = sum(len(s) for s in texts)          
        articleLength.append(length if length != 0 else 0)
    
    return context, articleLength

#### 分詞&計算詞頻

In [6]:
STOPWORD_PATH = f'/Users/{user_name}/Downloads/{file_dir}/stopwords-master/baidu_stopwords.txt'
__stopwords = list("\xa0\r\n\t\f\v ") + list(enPunc) + list(zhPunc)
__converter = OpenCC("tw2sp")

with open(STOPWORD_PATH, "r") as f:
    lines = f.read().splitlines()
    __stopwords.extend(lines)
    for line in lines:
        __stopwords.append(__converter.convert(line))

In [7]:
__valid_tags = ["n", "nz", "v", "vd", "vn", "a", "ad", "an", "d", "PER", "LOC", "ORG", "TIME"]
__lac = LAC(mode="lac")
__lac.load_customization(f'/Users/{user_name}/Downloads/{file_dir}/data/custom_dict.txt')

In [8]:
def __get_named_entities(texts):
    """取得文章命名實體
    
    :param texts: 所有網頁文章(list)
    :return for_target_page(目標頁所需之命名實體), for_info_extract(關鍵資訊擷取所需之命名實體)
    """
    for_target_page, for_info_extract = [], []
    for idx, text in enumerate(texts):
        try:
            result: list[(str, str)] = []
            if isinstance(text, (str, list)) and len(text):
                try:
                    lac_result = __lac.run(text if isinstance(text, list) else [text])
                except:
                    lac_result = __lac.run([" ".join(text)]) if isinstance(text, list) else []
                finally:
                    for words, tags in lac_result:
                        result.extend((words[idx], tags[idx]) for idx in range(len(words)) if len(words[idx])>1)
#             print(result) # [('中國國防科技', 'ORG'), ('信息', 'n'), ('中心', 'n'),....

            _for_target_page = [
                # 篩選出屬於合法詞性類別標籤且不包含在斷詞中的分詞
                word for word, tag in result
                if tag in __valid_tags
                and word not in __stopwords
                and not re.search(r"[0-9]", word)
            ]
            for_target_page.append(_for_target_page)

            _for_info_extract = [
                # 篩選出關鍵詞彙(客製化需求欄位，未來來源可能會調整)
                word for word, tag in result
                if tag in ["ORG", "PER", "nz", "an", "vn"]
                and word not in __stopwords
                and not re.search(r"[0-9]", word)
            ]
            for_info_extract.append(_for_info_extract)
            
        except Exception as e:
            for_target_page.append([])
            for_info_extract.append([])
    return for_target_page, for_info_extract

#### 計算tfidf

In [11]:
def __get_tfidfs(counters, top):
    """取得各篇文章的關鍵字及其透過TF-IDF加權分數的前20名
    
    :param counters: 各篇文章詞頻
    :param top: 前?名
    :return 各篇文章的前?名關鍵詞(dataframe)
    """
    def tf(word, counter):
        return counter[word] / sum(counter.values())

    def idf(word, counters):
        return math.log2(len(counters) / sum(1 for counter in counters if word in counter))

    data = []
    for idx, counter in enumerate(counters):
        data.extend([idx, word, round(score, 5)] for word, score in sorted([
            (word, tf(word, counter) * idf(word, counters)) for word in counter
        ], key=lambda x: x[1], reverse=True)[:top])
    
    return pd.DataFrame(data, columns=["idx", "word", "score"])

### 排序模組

#### 匯入自行訓練的詞向量並進行增量訓練

In [14]:
MODEL_PATH = f'/Users/{user_name}/Downloads/{file_dir}/model/api_version/word2vec.model'

def __get_trained_model(query, corpus):
    """詞向量增量訓練

    :param query(str): 搜尋詞
    :param corpus: 語料庫，預期帶入文章分詞結果集合
    :return: 增量訓練後的模型
    """
    if os.path.isfile(MODEL_PATH): #該路徑是否為文件
        model = Word2Vec.load(MODEL_PATH)
    else:
        return None

    words, tags = __lac.run(query)

    model.build_vocab(corpus, update=True, trim_rule=lambda word, count, min_count:
                      gensim.utils.RULE_KEEP if word in words else gensim.utils.RULE_DEFAULT)
    model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)
    model.save(MODEL_PATH)

    return model

In [33]:
### 匯入github下載之詞向量
# file_name = 'merge_sgns_bigram_char300.txt'
# with open(f'/Users/{user_name}/Downloads/{file_dir}/model/chinese_word_embedding/{file_name}') as f:
#     lines = f.readlines()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### 取得搜尋詞詞向量

In [17]:
def __get_vector(query, model):
    """取得搜尋詞及其向量

    若詞向量中有此搜尋詞，則取得該詞彙之向量，
    若詞向量中無此搜尋詞，則進行分詞，再尋找該詞彙之向量

    :param query(str): 搜尋詞
    :param model(Word2Vec): 詞向量的相關模型
    :return: vector
    """
    try:
        vector = model.wv.vectors[model.wv.key_to_index[query]]
    except:
        words, tags = __lac.run(query)
        vectors = [
            model.wv.vectors[model.wv.key_to_index[word]]
            for word in words if word not in __stopwords
        ]
        vector = sum(vectors) / len(vectors)
    return vector

#### 取得文件向量並與搜尋詞比較相似性

In [34]:
def __get_similarity(google_search_rs, top, query):
    contexts, articleLength = __get_content_texts(google_search_rs)
    for_target_page, for_info_extract = __get_named_entities(contexts)
    all_doc_count = [Counter(for_target_page[i]) for i in range(len(for_target_page))]
    tfidfs = __get_tfidfs(all_doc_count,top) #所有網頁內容分詞的tfidfs，return dataframe
    model = __get_trained_model(query, contexts) #詞向量模型
    query_vector = __get_vector(query, model) #搜尋詞的詞向量
    print(query_vector)
    
    vector = np.zeros(250)
    distance, words = [], []
    for idx in range(len(google_search_rs)):
        for row in tfidfs.loc[tfidfs["idx"] == idx].itertuples():
            try:
                vector += model.wv.vectors[model.wv.key_to_index[row.word]] * row.score
            except:
                continue
            else:
                words.append(row.word)
        else:
            total = len(words)
            distance.append(math.dist(vector if 0 == total else (vector / total), query_vector)) #各篇文章與搜尋詞的相似度

    for sort, idx in enumerate(
        sorted(range(len(google_search_rs)), key=lambda idx: distance[idx] 
               if 400 < articleLength[idx] else 100, reverse=False)):
        print("Article %d: %s"%(sort+1,google_search_rs[idx]))

## Info Extraction: V 1.2 (Api Version)

In [25]:
def __get_keywords(keywords, top):
    keyword_count = [Counter(keywords[i]) for i in range(len(keywords))]
    keyword_tfidfs = __get_tfidfs(keyword_count,top)
    return keyword_tfidfs

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Target Page Demo

In [35]:
google_search_rs = pd.read_excel(f'/Users/{user_name}/Downloads/{file_dir}/data/try_1112.xlsx').url.to_list() 
__get_similarity(google_search_rs, 15, "振华数据王雪峰情报")

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.epochtimes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.abc.net.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectio

('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'news.ltn.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.soundofhope.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bast.net.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/con

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.pttweb.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'news.chinanewscenter.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rightpoint.site'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/

[-0.5048549  -0.26320392  0.12253246  0.08631005  0.92290974 -0.05221258
  0.03147755 -0.08807772  1.1688981   0.41413298  0.4717567   0.16057268
  1.1779543   0.00453309 -0.07957526 -0.77915907  0.17196967  0.06559723
 -0.17994893  0.5717421   1.2732385   0.22441833 -1.2228162  -0.66108686
 -0.3419098  -0.33171663  0.55128074 -0.23075433 -0.01074704  0.20768262
 -0.0668118   0.59502393 -0.30634293 -0.13763218 -0.6657718   0.11595028
 -0.6966179  -0.44222078 -0.43109298 -0.06324557  0.19331992  0.17481901
  0.33570865 -0.23594707 -0.20033725 -0.66584784 -0.48221943 -0.18506174
 -0.22726141  0.32776988 -0.14923026  0.26352087  0.01926862 -0.9084549
  0.2628064   0.14000465  0.08180525 -0.05704756  0.09626955  0.95100385
  0.7172274   0.18072854  0.6179483  -0.3625981  -0.07808066  1.0485755
  0.20108831 -0.22731733  0.09919318  0.10227898  0.12025154  0.39510015
 -0.39845762  0.86709857  0.57398444 -0.33889708  0.02021452 -1.0293691
 -0.5285217   0.25187746 -0.26742145  0.48209813  0.30

## Info Extraction Demo

In [31]:
contexts, articleLength = __get_content_texts(google_search_rs)
for_target_page, for_info_extract = __get_named_entities(contexts)
df = __get_keywords(for_info_extract, 15)
df[df.idx==15]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,idx,word,score
211,15,鲍尔丁,0.18554
212,15,运作,0.15294
213,15,摄像,0.14110
214,15,红枫林新闻网,0.14110
215,15,信息技术有限公司,0.12370
216,15,振华数据,0.11844
217,15,中国大陆公司,0.11610
218,15,数据库,0.11054
219,15,中国共产党。公司,0.10147
220,15,美国川普政府,0.10147
